# Predicciones sobre intersecciones de la ciudad

Se trabajará con sobre el df DAM_toPredict con las predicciones del mejor modelo seleccionado para la construcción del df DAM_predictions el cual finalmente será unido en un df final que contenga todos los datos de intersecciones y puntos verdes. Sobre este df se realizará una visualización de datos geoespaciales en un nuevo notebook configurado con un entorno geopandas (PVR_DAM_mapa).

INSERTAR IMAGEN

In [128]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


### Data connection

In [129]:
PVR_DAM_geodata = pd.read_csv('Data/PVR_DAM_geodata.csv', delimiter=';')


In [130]:
DAM_to_predict = PVR_DAM_geodata[PVR_DAM_geodata['punto_tipo']
                                 == 'Intersercción']


### Data cleaning

In [131]:
DAM_to_predict.dtypes


ID                     object
punto_tipo             object
poblacion             float64
hogares               float64
desocupado            float64
0-14                  float64
65_mas                float64
inicial               float64
primario              float64
egb                   float64
secundario            float64
polimodal             float64
superior              float64
universitario         float64
post_universitario    float64
lee_y _esc            float64
NoLee_ni_e            float64
hacinamiento          float64
sin_agua              float64
sin_cloaca            float64
densidad_comercial    float64
microzonificacion      object
macrozonificacion      object
x                     float64
y                     float64
index                   int64
dtype: object

In [132]:
display(DAM_to_predict)


,ID,punto_tipo,poblacion,hogares,desocupado,0-14,65_mas,inicial,primario,egb,...,NoLee_ni_e,hacinamiento,sin_agua,sin_cloaca,densidad_comercial,microzonificacion,macrozonificacion,x,y,index
74,I1,Intersercción,845.0,253.0,36.0,186.0,122.0,27.0,190.0,56.0,...,42.0,2.0,0.0,3.0,8.0,Comercial,Altamente urbano,-68.832689,-32.862203,74
75,I2,Intersercción,958.0,287.0,37.0,207.0,143.0,29.0,174.0,82.0,...,45.0,3.0,0.0,7.0,8.0,NaN,NaN,-68.832652,-32.862198,75
76,I3,Intersercción,845.0,253.0,36.0,186.0,122.0,27.0,190.0,56.0,...,42.0,2.0,0.0,3.0,7.0,Residencial urbano,Urbano,-68.833496,-32.862253,76
77,I4,Intersercción,845.0,253.0,36.0,186.0,122.0,27.0,190.0,56.0,...,42.0,2.0,0.0,3.0,4.0,Residencial urbano,Urbano,-68.834222,-32.862308,77
78,I5,Intersercción,845.0,253.0,36.0,186.0,122.0,27.0,190.0,56.0,...,42.0,2.0,0.0,3.0,3.0,Residencial urbano,Urbano,-68.834852,-32.862354,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,I2382,Intersercción,340.0,146.0,13.0,31.0,66.0,7.0,26.0,8.0,...,8.0,0.0,1.0,3.0,0.0,Especial Parque Gral. San Marti­n,Especial,-68.865545,-32.901277,2455
2456,I2383,Intersercción,5.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,...,2.0,0.0,0.0,0.0,NaN,Especial Parque Gral. San Marti­n,Especial,-68.866127,-32.901451,2456
2457,I2384,Intersercción,5.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,...,2.0,0.0,0.0,0.0,NaN,Especial Parque Gral. San Marti­n,Especial,-68.866127,-32.901451,2457
2458,I2385,Intersercción,5.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,...,2.0,0.0,0.0,0.0,NaN,Especial Parque Gral. San Marti­n,Especial,-68.866127,-32.901451,2458


In [133]:
DAM_to_predict.isna().sum()


ID                      0
punto_tipo              0
poblacion              11
hogares                11
desocupado             11
0-14                   11
65_mas                 11
inicial                11
primario               11
egb                    11
secundario             11
polimodal              11
superior               11
universitario          11
post_universitario     11
lee_y _esc             11
NoLee_ni_e             11
hacinamiento           11
sin_agua               11
sin_cloaca             11
densidad_comercial    404
microzonificacion     101
macrozonificacion     101
x                       0
y                       0
index                   0
dtype: int64

In [134]:
DAM_to_predict.dropna(axis=0, inplace=True)


/var/folders/bs/_j3f29351c5cry0yc5phvjj40000gn/T/ipykernel_37769/3381475953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DAM_to_predict.dropna(axis=0, inplace=True)


In [135]:
DAM_to_predict.isna().sum()


ID                    0
punto_tipo            0
poblacion             0
hogares               0
desocupado            0
0-14                  0
65_mas                0
inicial               0
primario              0
egb                   0
secundario            0
polimodal             0
superior              0
universitario         0
post_universitario    0
lee_y _esc            0
NoLee_ni_e            0
hacinamiento          0
sin_agua              0
sin_cloaca            0
densidad_comercial    0
microzonificacion     0
macrozonificacion     0
x                     0
y                     0
index                 0
dtype: int64

### Data preparation

#### Feature selection

In [136]:
DAM_to_predict_filterd = DAM_to_predict[
    [
        'poblacion',
        '0-14',
        'primario',
        'secundario',
        'polimodal',
        'lee_y _esc',
        'NoLee_ni_e'
    ]
]


### Predictions

In [138]:
# Llamar a objeto pipeline
with open('./Models/logist-regresion.pkl', 'rb') as f:
    modelo_load = pickle.load(f)

with open('./Models/logist-regresion-scaler.pkl', 'rb') as s:
    scaler_load = pickle.load(s)

scaled_case = scaler_load.fit_transform(pd.DataFrame(DAM_to_predict_filterd))
clasificacion = modelo_load.predict(scaled_case)


In [139]:
df = DAM_to_predict.copy()
df['clasificacion'] = clasificacion
DAM_to_predict_filterd['clasificacion'] = clasificacion
DAM_to_predict_filterd['ID'] = DAM_to_predict['ID']
DAM_to_predict_filterd['x'] = DAM_to_predict['x']
DAM_to_predict_filterd['y'] = DAM_to_predict['y']
DAM_to_predict_filterd['punto_tipo'] = DAM_to_predict['punto_tipo']


/var/folders/bs/_j3f29351c5cry0yc5phvjj40000gn/T/ipykernel_37769/1729495590.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DAM_to_predict_filterd['clasificacion'] = clasificacion
/var/folders/bs/_j3f29351c5cry0yc5phvjj40000gn/T/ipykernel_37769/1729495590.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DAM_to_predict_filterd['ID'] = DAM_to_predict['ID']
/var/folders/bs/_j3f29351c5cry0yc5phvjj40000gn/T/ipykernel_37769/1729495590.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

### Output final

In [140]:
PVR_train_test = pd.read_csv('./PVR_train_test.csv', delimiter=',')


In [143]:
PVR_train_test.rename(columns={'Target_NoApto': 'clasificacion'}, inplace=True)
PVR_train_test_filterd = PVR_train_test[
    [
        'poblacion',
        '0-14',
        'primario',
        'secundario',
        'polimodal',
        'lee_y _esc', 
        'NoLee_ni_e',
        'x',
        'y',
        'ID',
        'clasificacion',
        'punto_tipo'
    ]
]


In [146]:
df_final = pd.concat([PVR_train_test_filterd, DAM_to_predict_filterd])
df_final.to_csv("./Data/PVR_DAM_all.csv", index=False, encoding='utf-8-sig')